In [8]:
import pandas as pd;
import numpy as np;
import time;
import sagemaker as sg;
import scipy;
import csv;
import xlearn as xl;
import random;
from tqdm import tqdm;


```
dataLink :  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
groupLens: https://grouplens.org/datasets/movielens/
readMe: http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html
```

### Summary
This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files links.csv, movies.csv, ratings.csv and tags.csv. More details about the contents and use of all these files follows.

This is a development dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available benchmark datasets if that is your intent.

This and other GroupLens data sets are publicly available for download at http://grouplens.org/datasets/.

In [12]:
#genome_scores = pd.read_csv('ml-latest/genome-scores.csv')
#genome_tags = pd.read_csv('ml-latest/genome-tags.csv')
#links = pd.read_csv('ml-latest-small/links.csv')
#movies = pd.read_csv('ml-1m/movies.dat')
#ratings = pd.read_csv('ml-1m/ratings.dat')
#users = pd.read_csv('ml-1m/users.dat')

In [17]:
rat_file = open('ml-1m/users.dat','r')
rat_file.readline()
rat_file.

'1::F::1::10::48067\n'

In [19]:
rat_file.readline()

'3::M::25::15::55117\n'

##### Preparing data in libsvm & libffm format

In [124]:
###Convert data to libsvm format


### reading & convert ratings file

def convert_ratings_to_fm(fin,fout,feature_index,rating_index,_model = "fm"):
    '''
    Input : ratings file with columns in the following order
            1) user_id
            2) movie_id
            3) rating
            4) timestamp - Ignoring this column for now
            
    Arguments : fin : input ratings file
                fout : output file name - column indices to be included
                column index containing the rating
                _model : ffm/fm
    
    
    Output:
            ratings matrix transformed to libsvm
    
    '''
    
    rat_file = open(fin,'r')  #input file
    text_file = open(fout,'w') #output file
    
    
    if _model=="ffm":
        add_field = 1
    else:
        add_field = 0
    
    
    #Initialize ::
    val0 = rat_file.readline();
    split_row0 = val0.split('::')
    datastring = ""  #stores the final string
    indx_cntr = 0
    d_field = {}

    ###User
    d_field[feature_index[0]] = {split_row0[0] : indx_cntr}
    indx_cntr = indx_cntr + 1
    ###movie
    d_field[feature_index[1]] = {split_row0[1] : indx_cntr}
    indx_cntr = indx_cntr + 1
    ###first string
    ###rating
    datastring += str(int(split_row0[rating_index]))
    ###user   
    datastring += "," + ("0" + ":") * add_field + str(d_field[0][split_row0[0]]) + ":" + "1"
    ###movie
    datastring += "," + ("1" + ":") * add_field + str(d_field[1][split_row0[1]]) + ":" + "1"
    
    datastring += "\n"
    text_file.write(datastring) 
    
    #iterate over all the lines
    for val in rat_file.readlines():

        #split each row
        split_row = val.split('::')
        #rating
        datastring = str(int(split_row[rating_index]))
        for col in feature_index: #ignoring timestamp, rating
           
            #if a new user/movie found, add it to dictionary
            if d_field[col].get(split_row[col],None) == None:
                d_field[col][split_row[int(col)]] = indx_cntr
                indx_cntr += 1

            datastring += "," + (str(col) + ":") * add_field + str(d_field[col][split_row[col]]) + ":" + "1"
        datastring += "\n"
        text_file.write(datastring)        
         
    text_file.close()     

##### Running a basic model

In [126]:
convert_ratings_to_fm("Test_new_v1.dat","fout",[0,1],2,_model = "fm")
fm_model = xl.create_fm()
fm_model.setTrain("./fout")
#fm_model.setValidate("./small_test.txt")
param = {'task':'reg', 'lr':0.2, 'lambda':0.002}

fm_model.fit(param, "./model.out")

In [128]:
convert_ratings_to_fm("Test_new_v1.dat","fout.2",[0,1],2,_model = "ffm")
ffm_model = xl.create_ffm()
ffm_model.setTrain("./testffm.txt")
#fm_model.setValidate("./small_test.txt")
param = {'task':'reg', 'lr':0.2, 'lambda':0.002}

ffm_model.fit(param, "./model.out")

##### Creating a train - test split 

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)

##### Benchmark it with Matrix factorization

In [ ]:
def rmse_k(data, k):
    indice = get_indices(data)
    ratios = [0.1, 0.3, 0.5, 0.7, 0.9]
    
    scores = dict()
    
    for ratio in tqdm(ratios):
        test = select_test(indice, ratio)
        new_data = mutate_ratings(data, test)
        pred = matrix_factorization(new_data,0.0002,1000, k = k)
        y_true, y_pred = fetch_ratings(data, pred, test)
        score = np.sqrt(mse(y_true, y_pred))
        scores[str(ratio)] = score
        
    return(scores)

In [ ]:
def select_test(indice, ratio):
    random.seed(2018)
    test = random.sample(indice, int(ratio*len(indice)))
    return(test)

test = select_test(indice, 0.1)


def mutate_ratings(ratings,test):
    rt = ratings.copy()
    for i in test:
        rt[i[0]][i[1]] = np.nan
    return (rt)

In [ ]:
def gen_test

In [ ]:
###Convert data to libsvm format


### reading ratings file
def convert_to_fm(filename,df,_type,features,_model = "fm",label = "rating"):
    '''
    Input : ratings file with columns in the following order
            1) user_id
            2) movie_id
            3) rating
            4) timestamp - Ignoring this column for now
    
    Output:
            ratings matrix transformed to libsvm
    
    '''
    
    
    #store fields for ffm
    df = df[features]
    
    
    #get total number of indices required
    field_list = list(df.columns)
    field_list.remove(label)
    
    #index count
    ind_cnt = -1
    
    #ind_dict is a dictionary of dictionary
    ind_dict = {}
    
    for col in  field_list:
        sub_dict = {}
        #maintains unique indices for unique values in each of the field
        sub_dict = dict(zip(df[col].unique(),map(lambda x : x + ind_cnt + 1,range(df[col].nunique()))))
        #sub_dict should have global indices against each unique value in the field
        
        ind_cnt += df[col].nunique()
        ind_dict[col] = sub_dict
    
    if _model=="ffm":
        add_field = 1
    else:
        add_field = 0
    
    nrow = df.shape[0]
    
    
    #write into a text file : ffm/fm processed data
    #val = f.split(",")
    
    text_file = open(filename,'w')
       
    for r in range(nrow):
        datastring = ""
        x = df.iloc[r].to_dict()
        datastring += str(int(x['rating']))
        x.pop('rating',None)
        
        for c in x.keys():
            datastring += "," + (c + ":") * add_field + str(int(ind_dict[c][x[c]])) + ":" + "1"
          
        datastring += "\n"
        text_file.write(datastring)
        
    text_file.close()